<a href="https://colab.research.google.com/github/SDS-AAU/SDS-master/blob/master/M3/notebooks/CNN_and_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using large pretrained models as your foundation
This notebook is based on https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/

Training large CNNs is costly (compute, time) and often we don't have enough data to bring these models to reasonable performance levels. This is where transfer-learning comes in.

Assuming that the early layers of a CNN learn very basic features (edges, lines etc.) we can consider a strategy, where we take a large pretrained model, discard the last couple of layers (where we would assume the more abstract information) and train it with our new data.

This is how you, for instance, could fine-tune a CNN model to some very specific medical imaging data where you simply cannot have so many examples.

In [1]:
# Let's start by downloading and exploring the data
!wget -qq https://storage.googleapis.com/sds-file-transfer/dataset.zip
# We need to unzip the data...and as you can see there is a lot
!unzip -qq dataset.zip

Keras has some great built-in applications - basically pretrained models with some nice functional overhead. We will start by loadeing VGG16 (a rather large model with many many many layers that has been trained on imagenet)

In [2]:
from keras.applications import VGG16
#Load the VGG model
image_size = 64
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

58892288/58889256 [==============================] - 0s 0us/step


The trick is, to "freeze" most of the initial layers and thereby preserve the majority of the information embedded in the model already. We will only train the last 2 Conv layers and also add a new dense layer as well as an output layer that fit's our purpose with the model - finding cats and dogs

In [3]:
# Freeze the layers except the last 2 layers
for layer in vgg_conv.layers[:-2]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f54d7aeaa20> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f54d71e7470> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f54d71e7780> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f54d71e7ba8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f54d71e8978> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f54a37084e0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f54a3708860> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f54a2ea5400> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f54a2eae278> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f54a2eae6a0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f54903cf198> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

In [4]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1025      
Total params: 16,813,889
Trainable params: 4,459,009
Non-trainable params: 12,354,880
_________________________________________________________________


In [5]:
# Importing the Keras-Image-Data-Generator <3 Keras ppl

from keras.preprocessing.image import ImageDataGenerator


# Defining the generators. Note, that we for obvious reasons only apply all of the generator magic to the training set.

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


test_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
# here, we specify the size of the images that we want (which needs to fit the input size of our network)
# We also set the batch size and the classification method (here binary)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [8]:
# And now we can train the network

model.fit_generator(training_set,
                         steps_per_epoch = 800,
                         epochs = 2,
                         validation_data = test_set,
                         validation_steps = 100)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
250/800 [========>.....................] - 35s 139ms/step - loss: 0.4961 - acc: 0.7680 - val_loss: 0.4045 - val_acc: 0.8165


In [9]:
from keras.preprocessing import image
import glob
import numpy as np

for i in glob.glob('dataset/test_set/dogs/*')[:10]:
  test_image = image.load_img(i, target_size = (64, 64))
  test_image = image.img_to_array(test_image)

  test_image = np.expand_dims(test_image, axis = 0)

  result = model.predict_classes(test_image)
  if result[0][0] == 1:
    print('dog')
  if result[0][0] == 0:
    print('cat')

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
dog
dog
dog
dog
dog
dog
dog
dog
dog
dog


In [10]:
# Evaluation returns loss and accuracy
model.evaluate_generator(test_set, steps=500)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.4044948220252991, 0.8165000081062317]